In [1]:
import cleverhans
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [18]:
# new Data preparation

# Training Dataset
data = pd.read_csv("./dataset/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv")

x_features = data[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max', 'category', 'subcategory']]

# Label Encoding the target columns
le = LabelEncoder()
x_features['category'] = le.fit_transform(x_features['category'])
x_features['subcategory'] = le.fit_transform(x_features['subcategory'])

x = x_features.values
x = preprocessing.scale(x)
y = data['attack'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

C:\Users\sumsu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\sumsu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [19]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(200, input_dim=x.shape[1], activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(200, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(np.unique(y))),
        # We seperate the activation layer to be able to access
        # the logits of the previous layer later
        tf.keras.layers.Activation(tf.nn.softmax)
        ])

model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
from keras.utils import np_utils
y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

model.fit(X_train, y_train_cat, epochs=10, validation_split=0.2)

Epoch 1/10
58697/58697 [==============================] - 480s 8ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 2.9071e-04 - val_accuracy: 1.0000
Epoch 2/10
58697/58697 [==============================] - 213s 4ms/step - loss: 3.6129e-04 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 3/10
58697/58697 [==============================] - 173s 3ms/step - loss: 3.2833e-04 - accuracy: 1.0000 - val_loss: 1.1227e-04 - val_accuracy: 1.0000
Epoch 4/10
58697/58697 [==============================] - 313s 5ms/step - loss: 4.8665e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 5/10
58697/58697 [==============================] - 304s 5ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 3.4646e-05 - val_accuracy: 1.0000
Epoch 6/10
58697/58697 [==============================] - 154s 3ms/step - loss: 4.1249e-04 - accuracy: 1.0000 - val_loss: 3.2963e-04 - val_accuracy: 1.0000
Epoch 7/10
58697/58697 [==============================] - 130s 2ms/step - loss: 

In [22]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)

print('Test accuracy:', test_acc)

18343/18343 [==============================] - 18s 1ms/step - loss: 1.8012e-05 - accuracy: 1.0000
Test accuracy: 0.9999948740005493


In [26]:
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
cm_org = confusion_matrix(y_test, y_pred)
print("*"*50)
print("Original confusion matrix")
print(cm_org)

18343/18343 [==============================] - 16s 893us/step
**************************************************
Original confusion matrix
[[    74      1]
 [     2 586887]]


In [28]:
# Import the attack
from cleverhans.future.tf2.attacks import fast_gradient_method

#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input, model.layers[-1].output)

epsilon = 0.1
adv_fgsm_x = fast_gradient_method(logits_model,
                                  X_test,
                                  epsilon,
                                  np.inf,
                                  targeted=False)

In [31]:
y_pred = model.predict(adv_fgsm_x)
y_pred=np.argmax(y_pred, axis=1)
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Attacked confusion matrix")
print(cm_adv)

18343/18343 [==============================] - 28s 2ms/step
********************
Attacked confusion matrix
[[    74      1]
 [     3 586886]]


In [32]:
print(adv_fgsm_x)

tf.Tensor(
[[ 0.25420835 -0.54503437 -1.58055229 ...  0.73174065 -0.83068184
  -0.86685956]
 [ 0.88066156 -0.41423933  0.71545303 ...  0.86690376  0.75246724
   0.84493152]
 [ 1.58192972 -0.81455091 -2.52355448 ...  0.81584392 -0.83068184
   0.84493152]
 ...
 [ 1.39726741 -1.10368447  0.46945246 ... -1.6224652   0.75246724
  -0.86685956]
 [-1.5093926  -0.52250187  0.71545303 ...  1.06422394 -0.83068184
   0.84493152]
 [-0.84419526 -1.20368447 -1.02955125 ... -1.5224652   0.65246724
  -0.96685956]], shape=(586964, 12), dtype=float64)
